# 1 数据预处理

## 1.1 一些基本情况的说明

In [1]:
#所使用的python版本与运行环境
import sys
print(sys.version)
print(sys.executable)

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
G:\Anaconda\python.exe


In [2]:
#所使用的包与其版本
import numpy as np
import pandas as pd
#import random
from warnings import warn
import sklearn
from sklearn.decomposition import PCA

In [3]:
print("numpy的版本为：",np.__version__)
print("pandas的版本为：",pd.__version__)
print("sklearn的版本为:",sklearn.__version__)

numpy的版本为： 1.19.5
pandas的版本为： 1.3.3
sklearn的版本为: 0.23.2


In [4]:
#所使用的数据
io=r'G:/Pythontask/final_py/depression.csv'
df=pd.read_csv(io,encoding='gbk')
print(df.shape) #原始数据的行数与列数
df.head()

(15874, 50)


,id,age,sex,prov,hukou,residence,edu,occupation,family_economic_condition,living_expenditure,...,b31,b32,b33,b34,b35,b36,b37,b38,b39,b310
0,11000118.0,104.0,2,11,1.0,1,0.0,4.0,4.0,2.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2.0
1,11000218.0,101.0,2,11,1.0,1,0.0,2.0,3.0,1.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,11000318.0,100.0,2,11,1.0,1,0.0,7.0,3.0,1.0,...,8.0,2.0,5.0,8.0,8.0,8.0,8.0,5.0,5.0,2.0
3,11000418.0,103.0,2,11,1.0,1,0.0,7.0,2.0,1.0,...,5.0,3.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,2.0
4,11000518.0,102.0,2,11,1.0,1,9.0,2.0,2.0,1.0,...,4.0,3.0,5.0,3.0,8.0,5.0,3.0,4.0,4.0,3.0


In [5]:
#原始数据的所有列名
list(df)

['id',
 'age',
 'sex',
 'prov',
 'hukou',
 'residence',
 'edu',
 'occupation',
 'family_economic_condition',
 'living_expenditure',
 'marriage',
 'coresidence',
 'housework',
 'taichi',
 'squaredance',
 'friends',
 'garden',
 'news',
 'pets',
 'cards',
 'tv',
 'social',
 'smoke',
 'alcohol',
 'exercise',
 'weight',
 'height',
 'fruit',
 'vegetable',
 'fish',
 'nut',
 'tea',
 'self_rated_health',
 'disease',
 'ADL1',
 'ADL2',
 'ADL3',
 'ADL4',
 'ADL5',
 'ADL6',
 'b31',
 'b32',
 'b33',
 'b34',
 'b35',
 'b36',
 'b37',
 'b38',
 'b39',
 'b310']

## 1.2 创建一个Questiondata类来对数据进行预处理

In [6]:
class RatioError(ValueError):
    """定义异常类"""
    pass

In [7]:
class Questiondata:
    """定义问卷数据类"""
    def __init__(self,df):
        self.data=df
    
    #查看某一列的不重复元素：
    def unique_data(colname):
        self.data[colname].unique()
    
    #定义一个方法用来替换变量
    def trans_data(self,dic,colname):
        for key,value in dic.items():
            key=int(key)
            self.data[colname].replace(value,key,inplace=True)
    
    #将身高和体重合成一个新的变量BMI
    def BMI(self):
        self.data['BMI']=self.data.weight/((self.data.height/100)**2)
        
    #将某一列中的缺失值全部替换成某一个特定的值
    def tran_na(self,colname,num):
        self.data[colname].fillna(num,inplace=True)
        
    #删除某一列大于某个特定值的行并输出删除的元素条数
    def drop_row(self,colname,num,inplace_type=True):
        index=self.data[self.data[colname]>=num].index
        self.data.drop(index,inplace=inplace_type)
        print("对于变量",colname," 删除的行数为：",len(index))
    
    #除了某一列，其它列均删除大于某个特定值的行并输出删除的元素条数
    def drop_row_mul(self,colnames,num,inplace_type=True):
        temp_list=list(filter(lambda x: x not in colnames,list(self.data)))
        for i in temp_list:
            self.drop_row(i,num,inplace_type)
        
    #将某几列的值相加生成一个新的变量
    def sum_col(self,col_list,newname):
        self.data[newname]=self.data[col_list].sum(axis=1)
    
    #求某一列缺失值的个数，且当缺失值个数超过1000的时候抛出警告
    def nan_num(self,colname):
        foo=self.data[colname]
        index=pd.isnull(self.data[colname])
        nan_num=len(foo[index])
        print(colname," 缺失值个数为：",nan_num)
        if nan_num>1000:
            warn("缺失值的个数超过1000了")
        
    #求除去某些列的缺失值的个数
    def nan_num_drop(self,colnames):
        temp_list=list(filter(lambda x: x not in colnames,list(self.data)))
        for i in temp_list:
            self.nan_num(i)  
    
    #删除所有的缺失值并输出删除的数据个数：
    def drop_nan_total(self):
        before=self.data.shape[0]
        self.data.dropna(inplace=True)
        after=self.data.shape[0]
        print("删除的缺失值个数为：",before-after)
    
    #利用主成分分析生成一个新的变量：
    def pca_new(self,colnames,new_name,pca_num=3):
        X=self.data.loc[:,colnames]
        model=PCA(n_components=pca_num,whiten=True)
        model.fit(X)
        components=model.components_
        explained_var=model.explained_variance_
        explained_var_ratio=model.explained_variance_ratio_
        print("各主成分的系数为：\n",components,'\n')
        print("各主成分的方差为：\n",explained_var,'\n')
        print("各主成分的方差占比为：\n",explained_var_ratio,'\n')
        if explained_var_ratio[0]<0.5:
            raise RatioError("Ratio less than 50%")
        else:
            self.data[new_name]=list(model.fit_transform(X)[:,0])
    
    #将某一列变量变成分类的：
    def cat_data(self,colname,order=True):
        self.data[colname]=pd.Categorical(self.data[colname],ordered=order) 
        
    #将除了某些列之外的列变成分类的：
    def cat_data_mult(self,colnames,order=False):
        temp_list=list(filter(lambda x: x not in colnames,list(self.data)))
        for i in temp_list:
            self.cat_data(i,order)  

## 1.3 数据的替换与类别的整合

In [8]:
#实例化
CLHLS=Questiondata(df)

In [9]:
#变量替换与整合
#将性别整合成两个水平，1为男性，0为女性
base_dic={'0':2}
CLHLS.trans_data(base_dic,colname='sex')

#将省份整合成4个水平
prov_dic={'1':[11,12,14,13,21,22,23,61],
          '2':[31,32,33,34,35,36,37],
          '3':[41,42,43],
          '4':[44,45,46,50,51]}
CLHLS.trans_data(prov_dic,colname='prov')

#将户口整合成两个水平，1为城市，0为农村
CLHLS.trans_data(base_dic,colname='hukou')

#将教育水平整合成3个水平，1为初等水平，2为中等水平，3为高等水平
edu_dic={'2':list(range(1,7)),
         '3':list(range(7,88)),
         '1':0}
CLHLS.trans_data(edu_dic,colname='edu')

#将职业整合成2个水平
occupation_dic={'1':[0,1],
           '0':list(range(2,9))}
CLHLS.trans_data(occupation_dic,colname='occupation')
CLHLS.tran_na('occupation',0)

#将生活支出整合成2个水平
CLHLS.trans_data(base_dic,colname='living_expenditure')

#将婚姻状况整合成2个水平
mar_dic={'0':list(range(2,6))+[8]}
CLHLS.trans_data(mar_dic,colname='marriage')

#将社会活动整个成3个水平
act_dic={'2':[2,3,4],
        '3':1,
        '1':5}
for i in ['housework','taichi','squaredance','friends','garden','news','pets','cards','tv','social']:
    CLHLS.trans_data(act_dic,colname=i)
    
#将吸烟、饮酒、锻炼整合成2个水平
base_dic={'0':2}
for i in ['smoke','alcohol','exercise']:
    CLHLS.trans_data(base_dic,colname=i)
    
#将身高和体重合成一个新的变量BMI
CLHLS.BMI()

#将水果、蔬菜合成两个水平，1为经常吃，0为偶尔或者很少
fruit_dic={'0':list(range(2,5))}
for i in ['fruit','vegetable']:
    CLHLS.trans_data(fruit_dic,colname=i)

#将鱼和坚果合成两个水平，1为经常吃，0为偶尔或者很少
fish_dic={'1':[1,2],
          '0':list(range(3,6))}
for i in ['fish','nut']:
    CLHLS.trans_data(fish_dic,colname=i)
    
#将茶整合成10个水平，将所有的缺失值替换成10不喝茶
CLHLS.tran_na('tea',10)

#将健康自评整合成2个水平
health_dic={'1':[1,2],
           '0':[3,4,5]}
CLHLS.trans_data(health_dic,colname='self_rated_health')

#将疾病整合成3个水平，1为有慢性病，2为其它疾病，3为没有疾病
disease1=[-2,3,4,5,11,14,17,19,20,24]
disease2=list(filter(lambda x: x not in disease1+[1,2],list(range(-2,26))))
disease_dic={'-2':2,
             '-1':1,
            '1':disease1,
            '2':disease2}
CLHLS.trans_data(disease_dic,colname='disease')
CLHLS.tran_na('disease',3)

#将生活能力整合成2类，1为不需要他人帮助，0为需要他人帮助
ADL_dic={'0':[2,3]}
for i in ["ADL"+str(x) for x in range(1,7)]:
    CLHLS.trans_data(ADL_dic,colname=i)
    
#将抑郁量表重新进行度量，重新整合成3个等级的分数
dep1_dic={'0':[1],'1':[2,3],'2':4,'3':5}
positive=['b35','b37','b310']
for i in positive:
    CLHLS.trans_data(dep1_dic,colname=i)
    
dep2_dic={'-1':1,'0':5,'1':[3,4],'3':[-1]}
negative=list(filter(lambda x: x not in positive,["b3"+str(x) for x in range(1,11)]))
for i in negative:
    CLHLS.trans_data(dep2_dic,colname=i)

## 1.4 对于缺失值的处理

由于我们的分析目标是看老年人的各种活动与其抑郁程度的关系，因此那些抑郁程度有缺失值的样本我们必须进行删除。

In [10]:
#删除所有关于抑郁程度单项问题中得分大于等于8的行
depression_list=["b3"+str(x) for x in range(1,11)]
for i in depression_list:
    CLHLS.drop_row(i,8)

对于变量 b31  删除的行数为： 1722
对于变量 b32  删除的行数为： 379
对于变量 b33  删除的行数为： 120
对于变量 b34  删除的行数为： 104
对于变量 b35  删除的行数为： 371
对于变量 b36  删除的行数为： 36
对于变量 b37  删除的行数为： 313
对于变量 b38  删除的行数为： 30
对于变量 b39  删除的行数为： 48
对于变量 b310  删除的行数为： 6


另外，一般情况下变量（除了id,年龄，身高，体重BMI，以及饮茶）中等于8（或者88）的为“不知道”，等于9（或者99）的为“缺失”，由于这些缺失值不多，我们也可以将其删除。

In [11]:
#删除基本的变量中得分大于等于8的行
CLHLS.drop_row_mul(['tea','age','id','weight','height','BMI']+depression_list,8,inplace_type=True)

对于变量 sex  删除的行数为： 0
对于变量 prov  删除的行数为： 0
对于变量 hukou  删除的行数为： 0
对于变量 residence  删除的行数为： 0
对于变量 edu  删除的行数为： 18
对于变量 occupation  删除的行数为： 0
对于变量 family_economic_condition  删除的行数为： 22
对于变量 living_expenditure  删除的行数为： 0
对于变量 marriage  删除的行数为： 2
对于变量 coresidence  删除的行数为： 0
对于变量 housework  删除的行数为： 1
对于变量 taichi  删除的行数为： 1
对于变量 squaredance  删除的行数为： 1
对于变量 friends  删除的行数为： 0
对于变量 garden  删除的行数为： 1
对于变量 news  删除的行数为： 4
对于变量 pets  删除的行数为： 2
对于变量 cards  删除的行数为： 0
对于变量 tv  删除的行数为： 2
对于变量 social  删除的行数为： 3
对于变量 smoke  删除的行数为： 0
对于变量 alcohol  删除的行数为： 1
对于变量 exercise  删除的行数为： 1
对于变量 fruit  删除的行数为： 1
对于变量 vegetable  删除的行数为： 1
对于变量 fish  删除的行数为： 0
对于变量 nut  删除的行数为： 2
对于变量 self_rated_health  删除的行数为： 11
对于变量 disease  删除的行数为： 41
对于变量 ADL1  删除的行数为： 1
对于变量 ADL2  删除的行数为： 5
对于变量 ADL3  删除的行数为： 1
对于变量 ADL4  删除的行数为： 1
对于变量 ADL5  删除的行数为： 0
对于变量 ADL6  删除的行数为： 0


In [12]:
BMI_list=['weight','height']
for i in BMI_list:
    CLHLS.drop_row(i,500)

对于变量 weight  删除的行数为： 311
对于变量 height  删除的行数为： 119


现在我们的缺失值只剩下了nan项，我们需要看一下它们在每个变量中的分布情况，如果数量不多且对数据整体没有影响，我们可以直接删除，如果有特殊情况我们需要按需处理。因此这里在类的nan_num_drop方法中我们设置了警告，当缺失的样本个数超过1000的时候就会抛出警告。

In [13]:
#为了方便后续对于数据的重新分析，我们这里将至此处理的数据保存到一个暂时的新questiondata实例中
temp_CLHLS=Questiondata(CLHLS.data.copy())
temp_CLHLS.data.drop('edu',axis=1, inplace=True)

In [14]:
#查看除了抑郁程度的所有列中缺失值的个数
CLHLS.nan_num_drop(depression_list)
#temp_CLHLS.nan_num_drop(depression_list)

id  缺失值个数为： 0
age  缺失值个数为： 8
sex  缺失值个数为： 0
prov  缺失值个数为： 0
hukou  缺失值个数为： 136
residence  缺失值个数为： 0
edu  缺失值个数为： 1846
occupation  缺失值个数为： 0
family_economic_condition  缺失值个数为： 180
living_expenditure  缺失值个数为： 180
marriage  缺失值个数为： 221
coresidence  缺失值个数为： 272
housework  缺失值个数为： 150
taichi  缺失值个数为： 182
squaredance  缺失值个数为： 179
friends  缺失值个数为： 174
garden  缺失值个数为： 143
news  缺失值个数为： 139
pets  缺失值个数为： 147
cards  缺失值个数为： 138
tv  缺失值个数为： 138
social  缺失值个数为： 237
smoke  缺失值个数为： 225
alcohol  缺失值个数为： 293
exercise  缺失值个数为： 275
weight  缺失值个数为： 144
height  缺失值个数为： 156
fruit  缺失值个数为： 126
vegetable  缺失值个数为： 120
fish  缺失值个数为： 240
nut  缺失值个数为： 258
tea  缺失值个数为： 0
self_rated_health  缺失值个数为： 130
disease  缺失值个数为： 0
ADL1  缺失值个数为： 156
ADL2  缺失值个数为： 207
ADL3  缺失值个数为： 145
ADL4  缺失值个数为： 205
ADL5  缺失值个数为： 155
ADL6  缺失值个数为： 376
BMI  缺失值个数为： 170


G:\Anaconda\lib\site-packages\ipykernel_launcher.py:47: UserWarning: 缺失值的个数超过1000了


- 包含教育变量的时候抛出了警告，可以看出缺失值最多的变量为教育。但是由于教育水平暂无合适的填补方法，我们只能暂时先将教育这个变量进行删除。在后续的分析中，如果教育水平这个变量没有特别重要的作用我们可以将其删除，然后将分析步骤重复一遍。这也是为什么我们之前定义一个新实例的意义所在。

In [15]:
CLHLS.drop_nan_total()
#temp_CLHLS.drop_nan_total()

删除的缺失值个数为： 3226


In [16]:
#删去了全部的缺失值之后现在数据的维数
CLHLS.data.shape
#temp_CLHLS.data.shape

(8966, 51)

## 1.5 增添新的变量

In [17]:
#对抑郁程度生成一个新的得分变量
col_list=["b3"+str(x) for x in range(1,11)]
CLHLS.sum_col(col_list,newname='depression')
#temp_CLHLS.sum_col(col_list,newname='depression')

In [18]:
#主成分分析
name_list=["occupation","family_economic_condition","living_expenditure"]
CLHLS.pca_new(colnames=name_list,pca_num=3,new_name='social_eco')
#temp_CLHLS.pca_new(colnames=name_list,pca_num=3,new_name='social_eco')

各主成分的系数为：
 [[-0.13191745  0.95746645 -0.25662382]
 [ 0.99027107  0.1157275  -0.07726812]
 [-0.0442832  -0.26432016 -0.9634178 ]] 

各主成分的方差为：
 [0.43293479 0.10130739 0.08991079] 

各主成分的方差占比为：
 [0.69363572 0.1623118  0.14405248] 



这里我们参考了文献的一般研究方法，想利用PCA构建一个“经济水平”变量。从PCA的结果可以看出，第一个主成分解释了方差的近70\%，而且其构成也是很有参考意义的——这个主成分得分越高，说明经济水平越低：因为我们将“职业”这个变量重新定义为0为非白领，1为白领；家庭经济条件则是分成了5个得分——得分越低说明越富有，越高说明越穷；生活支出的0则是意味着收入不能满足基本生活需求，1为收入能满足基本生活需求。

## 1.6 改变变量类型

In [19]:
#改变需要将数据变成有序变量的列
order_list=['edu','family_economic_condition']+['housework','taichi','squaredance','friends','garden','news','pets','cards','tv','social']+["b3"+str(x) for x in range(1,11)]
for i in order_list:
    #temp_CLHLS.cat_data(i)
    CLHLS.cat_data(i)


#改变不需要将数据变成有序变量的列
norder_exp_list=order_list+['id','age','social_eco','weight','height','BMI','depression']
CLHLS.cat_data_mult(norder_exp_list)
#temp_CLHLS.cat_data_mult(norder_exp_list)

In [20]:
print(CLHLS.data.dtypes)
#print(temp_CLHLS.data.dtypes)

id                            float64
age                           float64
sex                          category
prov                         category
hukou                        category
residence                    category
edu                          category
occupation                   category
family_economic_condition    category
living_expenditure           category
marriage                     category
coresidence                  category
housework                    category
taichi                       category
squaredance                  category
friends                      category
garden                       category
news                         category
pets                         category
cards                        category
tv                           category
social                       category
smoke                        category
alcohol                      category
exercise                     category
weight                        float64
height      

到这里，我们的数据预处理部分总算是告一段落。接下来我们进入对于数据的分析部分。